In [1]:
from database_manager import load_database


load_database(
    "allergen30",
    "https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/9ygs9vhnpw-1.zip",
    "ab6e19d32f7490988ca77d600fc6f3df2e8648365c4c92ced8c1b462c01d9d9f"
)

2024-11-05,13:29:44.603 INFO: Dataset allergen30 SHA256 verification successful!
2024-11-05,13:29:50.077 INFO: Dataset allergen30 available in /home/pseusys/Documents/repositories/imagine_food/datasets/allergen30!
